In [1]:
import requests
import pandas as pd
import io
import time
import datetime
import numpy as np
import json
import re

In [4]:
TWSE_list = pd.DataFrame()
def Crawler_TWSE_List():
  global TWSE_list
  url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
  page = requests.get(url)
  TWSE_list = pd.read_html(page.text)[0]
  TWSE_list.columns = TWSE_list.iloc[0]
  end = 0
  for i in range(0,len(TWSE_list)):
    if (TWSE_list.at[i, '有價證券代號及名稱'][0:4] == '上市認購'):
        end = i
  TWSE_list = TWSE_list.iloc[2:end]
  TWSE_list = TWSE_list[['有價證券代號及名稱']]
  TWSE_list  = TWSE_list.rename(columns={'有價證券代號及名稱': 'Stockcode'})
  TWSE_list.reset_index(drop=True, inplace=True)
  for i in range (0,len(TWSE_list)):
    TWSE_list.at[i, 'Stockcode'] = TWSE_list.at[i, 'Stockcode'][5:]+"("+TWSE_list.at[i, 'Stockcode'][0:4]+")"

In [5]:
TPEx_list = pd.DataFrame()
def Crawler_TPEx_List():
  global TPEx_list
  url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=4'
  page = requests.get(url)
  TPEx_list = pd.read_html(page.text)[0]
  TPEx_list.columns = TPEx_list.iloc[0]
  start = 0
  end = 0
  for i in range(0,len(TPEx_list)):
    if (TPEx_list.at[i, '有價證券代號及名稱'][0:4] == '股票'):
        start = i+1
    if (TPEx_list.at[i, '有價證券代號及名稱'][0:4] == '特別股'):
        end = i
  TPEx_list = TPEx_list.iloc[start:end]
  TPEx_list = TPEx_list[['有價證券代號及名稱']]
  TPEx_list = TPEx_list.rename(columns={'有價證券代號及名稱': 'Stockcode'})
  TPEx_list.reset_index(drop=True, inplace=True)
  for i in range (0,len(TPEx_list)):
    TPEx_list.at[i, 'Stockcode'] = TPEx_list.at[i, 'Stockcode'][5:]+"("+TPEx_list.at[i, 'Stockcode'][0:4]+")"

In [7]:
StockPrice_TWSE = pd.read_csv('Stock_TWSE.csv', dtype=str, header=[0,1], index_col=0)
StockPrice_TPEx = pd.read_csv('Stock_TPEx.csv', dtype=str, header=[0,1], index_col=0)

In [8]:
different_Tpex = None
different_TWSE = None
def Update_Stock_List():
  global different_Tpex
  global different_TWSE
  Crawler_TPEx_List()
  Crawler_TWSE_List()
  if list(dict.fromkeys(StockPrice_TPEx.columns.get_level_values(0)[1:])) == list(TPEx_list.iloc[:,0]):
    print('TPEx Pass')
  else:
    if (len(TPEx_list) > (len(dict.fromkeys(StockPrice_TPEx.columns.get_level_values(0)[1:])))):
        different_Tpex = set(TPEx_list.iloc[:,0]) - set(StockPrice_TPEx.columns.get_level_values(0)[1:])
        different_Tpex = list(different_Tpex)
        print("The different TPEx List:"+ different_Tpex)
        for i in range(0,len(different_Tpex)):
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Close'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Volume'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Foundation'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Yield'), None)
            StockPrice_TPEx.sort_index(axis = 1, level=0, inplace=True, sort_remaining = False)
            temp = StockPrice_TPEx.pop(('Date',       'Date'))
            StockPrice_TPEx.insert(0, ('Date',       'Date'), temp)
    else:
        different_Tpex = set(StockPrice_TPEx.columns.get_level_values(0)[1:]) - set(TPEx_list.iloc[:,0])
        different_Tpex = list(different_Tpex)
        print("The different TPEx List:"+ different_Tpex)
        for i in range(0,len(different_Tpex)):
            StockPrice_TPEx.drop(columns=[different_Tpex[i]], inplace=True)
            
  if list(dict.fromkeys(StockPrice_TWSE.columns.get_level_values(0)[1:])) == list(TWSE_list.iloc[:,0]):
    print('TWSE Pass')
  else:
    if (len(TWSE_list) > (len(dict.fromkeys(StockPrice_TWSE.columns.get_level_values(0)[1:])))):
        different_TWSE = set(TWSE_list.iloc[:,0]) - set(StockPrice_TWSE.columns.get_level_values(0)[1:])
        different_TWSE = list(different_TWSE)
        print("The different TPEx List:"+ different_TWSE)
        for i in range(0,len(different_TWSE)):
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Close'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Volume'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Foundation'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Yield'), None)
            StockPrice_TWSE.sort_index(axis = 1, level=0, inplace=True, sort_remaining = False)
            temp = StockPrice_TWSE.pop(('Date',       'Date'))
            StockPrice_TWSE.insert(0, ('Date',       'Date'), temp)
    else:
        different_TWSE = set(StockPrice_TWSE.columns.get_level_values(0)[1:]) - set(TWSE_list.iloc[:,0])
        different_TWSE = list(different_TWSE)
        print("The different TPEx List:"+ different_TWSE)
        for i in range(0,len(different_TWSE)):
            StockPrice_TWSE.drop(columns=[different_TWSE[i]], inplace=True)

In [9]:
def crawler(datetime):
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=csv&d='+ datetime +'&se=EW&s=0,asc,0'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if '代號' in text:
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    for i in range(0,len(test_df)):
        try:
            if (len(test_df.loc[i,'代號']) != 4):
                test_df.drop(i,inplace=True)
        except TypeError:
            if test_df.loc[i,'代號'] < 1200 and test_df.loc[i,'代號'] > 9999:
                test_df.drop(i,inplace=True)
    for i in range(0, len(test_df)):
        test_df.iloc[i,0] = str(test_df.iloc[i,1])+"("+ str(test_df.iloc[i,0])+")"
    test_df = test_df[['代號', '收盤 ', '成交股數  ']]
    test_df.iloc[:,2] = test_df.iloc[:,2].str.replace(",","").astype(float)    
    return test_df

def crawler_Foundation(datetime):
    url = 'https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&o=csv&se=EW&t=D&d='+ datetime +'&s=0,asc'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if '代號' in text:
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    if len(test_df) > 1:
        for i in range(0,len(test_df)):
            try:
                if (len(test_df.loc[i,'代號']) != 4):
                    test_df.drop(i,inplace=True)
            except TypeError:
                if test_df.loc[i,'代號'] < 1200 and test_df.loc[i,'代號'] > 9999:
                    test_df.drop(i,inplace=True)
        for i in range(0, len(test_df)):
            test_df.iloc[i,0] = str(test_df.iloc[i,1])+"("+ str(test_df.iloc[i,0])+")"
        try:
            test_df = test_df[['代號', '三大法人買賣超股數合計']]
        except KeyError:
            test_df = test_df[['代號', '三大法人買賣超股數']]
        test_df.reset_index(drop=True,inplace=False)
    else:
        del test_df
    test_df.iloc[:,1] = test_df.iloc[:,1].str.replace(",","").astype(float)
    return test_df

def crawler_Yield(datetime):
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/peratio_analysis/pera_result.php?l=zh-tw&o=csv&charset=UTF-8&d='+datetime+'&c=&s=0,asc'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
            if '股票代號' in text:
                initial_point = i
                break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    if (len(use_text) < 10):
        del test_df
    else:
        for i in range(0, len(test_df)):
            test_df.iloc[i,0] = str(test_df.iloc[i,1])+"("+ str(test_df.iloc[i,0])+")"
        test_df = test_df[['股票代號', '殖利率(%)']]
        test_df = test_df.rename(columns={"股票代號": '代號'})
    return test_df

#Day_TPEx = []
def trans_date(datetime):
#    global Day_TPEx
    d = datetime.day
    m = datetime.month
    y = datetime.year-1911
    if m < 10:
        if d < 10:
            Day_TPEx = (str(y)+'/0'+str(m)+'/0'+str(d))
        else:
            Day_TPEx = (str(y)+'/0'+str(m)+'/'+str(d))
    else:
        if d < 10:
            Day_TPEx = (str(y)+'/'+str(m)+'/0'+str(d))
        else:
            Day_TPEx = (str(y)+'/'+str(m)+'/'+str(d))
            
    return Day_TPEx

Crawler_Price = pd.DataFrame()
Crawler_Foundation = pd.DataFrame()
Crawler_Yield = pd.DataFrame()
def crawler_data(datetime):
    try:
        global Crawler_Price
        global Crawler_Foundation
        global Crawler_Yield
        Crawler_Price = crawler(trans_date(datetime))
        Crawler_Foundation = crawler_Foundation(trans_date(datetime))
        Crawler_Yield = crawler_Yield(trans_date(datetime))
        print('TPEx Crawler Successful')
    except UnboundLocalError:
        print('TPEx Crawler Fail')
    except KeyError:
        print('TPEx Crawler Fail')

In [10]:
def catch(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date='+datetime+'&type=ALLBUT0999&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['tables'][8]['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['tables'][8]['fields']])
    for i in range(0,len(test_df)):
        if ((len(test_df.loc[i,'證券代號'][0])) != 4):
            test_df.drop(i,inplace=True)
        elif int(test_df.loc[i,'證券代號'][0]) < 1100 or int(test_df.loc[i,'證券代號'][0]) > 9999:
            test_df.drop(i,inplace=True)
    temp = test_df[['證券代號', '證券名稱', '收盤價', '成交股數']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0],temp.columns[2][0],temp.columns[3][0]])
    for i in range(0, len(test_df)):
        test_df.iloc[i,0] = str(test_df.iloc[i,1])+"("+ str(test_df.iloc[i,0])+")"
    test_df = test_df[['證券代號', '收盤價', '成交股數']]
    test_df.iloc[:,2] = test_df.iloc[:,2].str.replace(",","").astype(float)
    return test_df

def catch_Foundation(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/fund/T86?date='+datetime+'&selectType=ALLBUT0999&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['fields']])
    for i in range(0,len(test_df)):
        if ((len(test_df.loc[i,'證券代號'][0])) != 4):
            test_df.drop(i,inplace=True)
        elif int(test_df.loc[i,'證券代號'][0]) < 1100 or int(test_df.loc[i,'證券代號'][0]) > 9999:
            test_df.drop(i,inplace=True)
    temp = test_df[['證券代號', '證券名稱', '三大法人買賣超股數']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0],temp.columns[2][0]])
    test_df.sort_values(by=['證券代號'], inplace=True)
    for i in range(0, len(test_df)):
        test_df.iloc[i,0] = str(test_df.iloc[i,1].split()[0])+"("+ str(test_df.iloc[i,0])+")"
    test_df = test_df[['證券代號', '三大法人買賣超股數']]
    test_df.iloc[:,1] = test_df.iloc[:,1].str.replace(",","").astype(float)
    return test_df

def catch_Yield(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date='+datetime+'&selectType=ALL&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['fields']])
    temp = test_df[['證券代號', '證券名稱', '殖利率(%)']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0],temp.columns[2][0]])
    for i in range(0, len(test_df)):
        test_df.iloc[i,0] = str(test_df.iloc[i,1])+"("+ str(test_df.iloc[i,0])+")"
    test_df = test_df[['證券代號', '殖利率(%)']]    
    return test_df
    
def catch_day(datetime):
    d = datetime.day
    m = datetime.month
    y = datetime.year
    if m < 10:
        if d < 10:
            Day_TWSE = str(y)+'0'+str(m)+'0'+str(d)
        else:
            Day_TWSE = str(y)+'0'+str(m)+str(d)
    else:
        if d < 10:
            Day_TWSE = str(y)+str(m)+'0'+str(d)
        else:
            Day_TWSE = str(y)+str(m)+str(d)
            
    return Day_TWSE

Catch_Price = pd.DataFrame()
Catch_Foundation = pd.DataFrame()
Catch_Yield = pd.DataFrame()
def catch_data(datetime):
    try:
        global Catch_Price
        global Catch_Foundation
        global Catch_Yield
        Catch_Price = catch(catch_day(datetime))
        Catch_Foundation = catch_Foundation(catch_day(datetime))
        Catch_Yield = catch_Yield(catch_day(datetime))
        print('TWSE Crawler Successful')
    except UnboundLocalError:
        print('TWSE Crawler Fail')
    except KeyError:
        print('TWSE Crawler Fail')

In [11]:
Crawler_Frame = pd.DataFrame()
Catch_Frame = pd.DataFrame()
def Get_New_Data(datetime):
    start = 0
    global Crawler_Frame
    global Crawler_Price
    global Crawler_Foundation
    global Crawler_Yield
    global Catch_Frame
    global Catch_Price
    global Catch_Foundation
    global Catch_Yield
    crawler_data(datetime)
    Crawler_Frame = pd.concat([Crawler_Price.set_index('代號'), Crawler_Foundation.set_index('代號').reindex(Crawler_Price.set_index('代號').index), Crawler_Yield.set_index('代號').reindex(Crawler_Price.set_index('代號').index)], axis=1)
    Crawler_Frame = Crawler_Frame.reset_index()
    catch_data(datetime)
    Catch_Frame = pd.concat([Catch_Price.set_index('證券代號'), Catch_Foundation.set_index('證券代號').reindex(Catch_Price.set_index('證券代號').index), Catch_Yield.set_index('證券代號').reindex(Catch_Price.set_index('證券代號').index)], axis=1)
    Catch_Frame = Catch_Frame.reset_index()

In [12]:
def Time_transfer(datetime):
  d = datetime.day
  m = datetime.month
  y = datetime.year
  if m < 10:
    if d < 10:
      Time = str(y)+'-0'+str(m)+'-0'+str(d)
    else:
      Time = str(y)+'-0'+str(m)+'-'+str(d)
  else:
    if d < 10:
      Time = str(y)+'-'+str(m)+'-0'+str(d)
    else:
      Time = str(y)+'-'+str(m)+'-'+str(d)
  return Time

In [13]:
def SortStockCode(Frame):
    test_list = []
    tmp = []
    for i in range(0,len(Frame)):
        test_list.append(int(re.search('\(([^)]+)',Frame[i]).group(1)))
        test_series = pd.Series(data=test_list)
        test_series = test_series.sort_values()
        tmp = list(test_series.index)
    return tmp

In [14]:
def Update_Stock_Price(datetime):
  temp = 0
  try:
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=csv&d='+trans_date(datetime)+'&se=EW&s=0,asc,0'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
      if '代號' in text:
        initial_point = i
        break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
  except NameError:
    print('No New Trading Data')
    temp = 1
                            
  if temp == 0:
    Update_Stock_List()
    Get_New_Data(datetime)
    global StockPrice_TWSE
    global StockPrice_TPEx
    temp1 = 0
    temp2 = 0
    if StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,0] == Time_transfer(datetime):
      print('TWSE Exists!')
      temp1 = 1
    #else:
      #StockPrice_TWSE = StockPrice_TWSE.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)
    
    if StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,0] == Time_transfer(datetime):
      print('TPEx Exists!')
      temp2 = 1
    #else:
      #StockPrice_TPEx = StockPrice_TPEx.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)
    '''
    if temp2 == 0:
      for i in range (0,len(Crawler_Frame)):
        for j in range (1, len(list(StockPrice_TPEx)),4):
          if (Crawler_Frame.loc[i,'代號'] == list(StockPrice_TPEx)[j][0]):
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j] = float(Crawler_Frame.iloc[i,1])
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = float(Crawler_Frame.iloc[i,2].replace(',',''))
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            except AttributeError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+2] = float(Crawler_Frame.iloc[i,3].replace(',',''))
            except ValueError:              
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+2] = None
            except AttributeError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+3] = float(Crawler_Frame.iloc[i,4])
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+3] = None
    
    if temp1 == 0:
      for x in range (0,len(Catch_Frame)):
        for y in range (1, len(list(StockPrice_TWSE)),4):
          if (Catch_Frame.loc[x,'證券代號'] == list(StockPrice_TWSE)[y][0]):
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y] = float(Catch_Frame.iloc[x,1])
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = float(Catch_Frame.iloc[x,2].replace(',',''))
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            except AttributeError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+2] = float(Catch_Frame.iloc[x,3].replace(',',''))
            except ValueError:              
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+2] = None
            except AttributeError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+3] = float(Catch_Frame.iloc[x,4])
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+3] = None
    '''
    AddingData = pd.DataFrame()
    if temp1 == 0:
      if set(Catch_Frame.iloc[:,0]) ^ set(StockPrice_TWSE.columns.get_level_values(0)[1:]) == set():
        SortAddingData = Catch_Frame.iloc[:,1:].values.ravel()
        tmp = np.insert(SortAddingData, 0, Time_transfer(datetime))
        tmp_data = pd.Series(data = tmp)
        StockPrice_TWSE = pd.concat([StockPrice_TWSE, pd.DataFrame(np.arange(len(StockPrice_TWSE.columns)).reshape(1,-1),columns=StockPrice_TWSE.columns)], ignore_index=True)
        StockPrice_TWSE.iloc[-1,:] = tmp_data
      else:
        Different = set(Catch_Frame.iloc[:,0]) ^ set(StockPrice_TWSE.columns.get_level_values(0)[1:])
        TempDifferent = Different.copy()
        AddingData = Catch_Frame.copy()
        for i in TempDifferent:
          try:
            AddingData = AddingData.drop(Catch_Frame.index[Catch_Frame["證券代號"] == i][0])
            Different.remove(i)
          except:
            AddingData = AddingData
        AddingData = pd.concat([AddingData,pd.DataFrame({'證券代號':list(Different)})], ignore_index=True)
        SortAddingData = AddingData.sort_values(by="證券代號", key=lambda x: np.argsort(SortStockCode(AddingData["證券代號"]))).iloc[:,1:].values.ravel()
        tmp = np.insert(SortAddingData, 0, Time_transfer(datetime))
        tmp_data = pd.Series(data = tmp)
        StockPrice_TWSE = pd.concat([StockPrice_TWSE, pd.DataFrame(np.arange(len(StockPrice_TWSE.columns)).reshape(1,-1),columns=StockPrice_TWSE.columns)], ignore_index=True)
        StockPrice_TWSE.iloc[-1,:] = tmp_data

    AddingData2 = pd.DataFrame()
    if temp2 == 0:
      if set(Crawler_Frame.iloc[:,0]) ^ set(StockPrice_TPEx.columns.get_level_values(0)[1:]) == set():
        SortAddingData2 = Crawler_Frame.iloc[:,1:].values.ravel()
        tmp2 = np.insert(SortAddingData2, 0, Time_transfer(datetime))
        tmp_data2 = pd.Series(data = tmp2)
        StockPrice_TPEx = pd.concat([StockPrice_TPEx, pd.DataFrame(np.arange(len(StockPrice_TPEx.columns)).reshape(1,-1),columns=StockPrice_TPEx.columns)], ignore_index=True)
        StockPrice_TPEx.iloc[-1,:] = tmp_data2
      else:
        Different2 = set(Crawler_Frame.iloc[:,0]) ^ set(StockPrice_TPEx.columns.get_level_values(0)[1:])
        TempDifferent2 = Different2.copy()
        AddingData2 = Crawler_Frame.copy()
        for i in TempDifferent2:
          try:
            AddingData2 = AddingData2.drop(Crawler_Frame.index[Crawler_Frame["代號"] == i][0])
            Different2.remove(i)
          except:
            AddingData2 = AddingData2
        AddingData2 = pd.concat([AddingData2,pd.DataFrame({'代號':list(Different2)})], ignore_index=True)
        SortAddingData2 = AddingData2.sort_values(by="代號", key=lambda x: np.argsort(SortStockCode(AddingData2["代號"]))).iloc[:,1:].values.ravel()
        tmp2 = np.insert(SortAddingData2, 0, Time_transfer(datetime))
        tmp_data2 = pd.Series(data = tmp2)
        StockPrice_TPEx = pd.concat([StockPrice_TPEx, pd.DataFrame(np.arange(len(StockPrice_TPEx.columns)).reshape(1,-1),columns=StockPrice_TPEx.columns)], ignore_index=True)
        StockPrice_TPEx.iloc[-1,:] = tmp_data2

    print('Finished!')  

In [15]:
Update_Stock_Price(datetime.datetime(2023, 5, 30, 10, 1, 55, 280233))

TPEx Pass
TWSE Pass
TPEx Crawler Successful
TWSE Crawler Successful
TWSE Exists!
TPEx Exists!
Finished!


In [2]:
print(pd.__version__)

2.0.0


In [1100]:
StockPrice_TWSE

Date 台泥(1101)                            亞泥(1102)           \
            Date    Close    Volume Foundation Yield    Close   Volume   
0     2016-01-04    26.15  10437306 -2717840.0  9.52    26.25  6852789   
1     2016-01-05    26.40   5382070  1819649.0  9.43    26.00  3967822   
2     2016-01-06    26.05   5154746  -135962.0  9.56    25.95  3736159   
3     2016-01-07    27.35  20289500  8650064.0  9.10    26.15  5483126   
4     2016-01-08    27.00  11915555 -3786818.0  9.22    26.00  5233511   
...          ...      ...       ...        ...   ...      ...      ...   
1801  2023-05-24    37.85  21681392 -6076747.0  1.32    44.35  4146061   
1802  2023-05-25    38.05  30414582 -4526019.0  1.31    44.00  5538911   
1803  2023-05-26    38.00  21233325  -689677.0  1.32    43.90  4926247   
1804  2023-05-29    38.15  26028333 -2975808.0  1.31    43.85  4356406   
1805  2023-05-30    38.30  28867665  1312784.0  1.31    44.05  4436916   

                      嘉泥(1103)  ... 三發地產(9946)       佳龍(9955)          \
     Foundation Yield    Close  ... Foundation Yield    Close  Volume   
0    -2582600.0  8.38     9.40  ...    11000.0  3.09    12.15   77112   
1     -390654.0  8.46     9.35  ...   -20000.0  3.13    12.20   74106   
2      -27353.0  8.48     9.27  ...   -39000.0  3.20    12.10   25100   
3     -142190.0  8.41     9.26  ...   -47365.0  3.42    12.20  124337   
4    -2929390.0  8.46     9.23  ...     2000.0  3.43    12.50  207000   
...         ...   ...      ...  ...        ...   ...      ...     ...   
1801 -1619925.0  5.19    20.20  ...    30000.0  2.90    26.30  625488   
1802 -3190003.0  5.23    20.00  ...     8000.0  2.80    25.65  681583   
1803 -2666687.0  5.24    19.85  ...    -7000.0  2.82    25.30  865858   
1804 -1919519.0  5.25    19.95  ...     6000.0  2.88    25.15  854295   
1805  -281703.0  5.22    20.00  ...     2000.0  2.88    24.90  560866   

                      世紀鋼(9958)                             
     Foundation Yield     Close    Volume Foundation Yield  
0        4000.0   0.0      9.95     56147    -7000.0  3.02  
1      -19000.0   0.0      9.95    101434   -20000.0  3.02  
2           NaN   0.0      9.84     71995        NaN  3.05  
3        2337.0   0.0      9.78    112830    -7000.0  3.07  
4      -23000.0   0.0      9.86    101907    39000.0  3.04  
...         ...   ...       ...       ...        ...   ...  
1801   -24000.0   0.0     98.60   4337318 -2393680.0  0.51  
1802  -203000.0   0.0     98.60   3381621  -892261.0  0.51  
1803  -265000.0   0.0    103.00   5554288  2254863.0  0.49  
1804   116000.0   0.0    109.50  10801916  2519442.0  0.46  
1805   -82000.0   0.0    109.50   4839576  -554174.0  0.46  

[1806 rows x 3901 columns]

In [1099]:
StockPrice_TPEx

Date 茂生農經(1240)                           其祥-KY(1258)            \
            Date      Close   Volume Foundation Yield       Close    Volume   
0     2016-01-04        NaN      NaN        NaN   NaN       39.50   51000.0   
1     2016-01-05        NaN      NaN        NaN   NaN       38.90  116000.0   
2     2016-01-06        NaN      NaN        NaN   NaN       38.05   85000.0   
3     2016-01-07        NaN      NaN        NaN   NaN       36.50  109000.0   
4     2016-01-08        NaN      NaN        NaN   NaN       37.30   59000.0   
...          ...        ...      ...        ...   ...         ...       ...   
1801  2023-05-24      46.00  20000.0        NaN  2.61       26.85   11000.0   
1802  2023-05-25      45.70  74000.0     1000.0  2.63       26.85   28000.0   
1803  2023-05-26      45.75  35000.0        0.0  2.62       26.85    8000.0   
1804  2023-05-29      45.40  29000.0    -2000.0  2.64       26.85   26000.0   
1805  2023-05-30      45.15  38000.0    -2000.0  2.66       26.85    6000.0   

                      安心(1259)  ...   皇田(9951)       邁達康(9960)           \
     Foundation Yield    Close  ... Foundation Yield     Close   Volume   
0           NaN  3.29     72.1  ...        NaN  3.33     19.00   2000.0   
1           NaN  3.34     70.9  ...        NaN  3.42     18.90   4000.0   
2           NaN  3.42     70.3  ...        NaN  3.48     18.70   2000.0   
3           NaN  3.56     69.1  ...        NaN  3.54     18.85  78000.0   
4           NaN  3.49     71.8  ...        NaN  3.41     18.75  83000.0   
...         ...   ...      ...  ...        ...   ...       ...      ...   
1801        NaN  0.00     77.2  ...     1000.0  4.45     30.90  33000.0   
1802        NaN  0.00     76.7  ...     5000.0  4.47     30.95  51000.0   
1803        NaN  0.00     76.4  ...   -46000.0  4.54     31.05  79000.0   
1804        NaN  0.00     76.8  ...   -44000.0  4.51     31.00  74000.0   
1805        NaN  0.00     76.0  ...    -3000.0  4.48     30.60  70000.0   

                      有益(9962)                           
     Foundation Yield    Close  Volume Foundation Yield  
0           NaN  4.21     7.25    1000        NaN  2.76  
1           NaN  4.23     7.17    2000        NaN  2.79  
2           NaN  4.28     7.06   35000        NaN  2.83  
3           NaN  4.24     7.07   30000        NaN  2.83  
4           NaN  4.27     7.06    1000        NaN  2.83  
...         ...   ...      ...     ...        ...   ...  
1801        NaN  7.12    19.75  580000    81000.0  9.11  
1802    -6000.0  7.11    19.50  456000   -32000.0  9.23  
1803        NaN  7.09    19.25  640000  -351000.0  9.35  
1804        0.0  7.10    19.25  254000     3000.0  9.35  
1805     1000.0  7.19    19.15  429000  -180000.0  9.40  

[1806 rows x 3249 columns]

In [1133]:
StockPrice_TWSE.to_csv('Stock_TWSE.csv')
StockPrice_TPEx.to_csv('Stock_TPEx.csv')

In [1134]:
MA_TWSE = pd.DataFrame(index=['7MA', '14MA', '30MA', '60MA', '90MA', '180MA', '360MA', '720MA', '1080MA'],columns=StockPrice_TWSE.columns.get_level_values(0)[1:].drop_duplicates())
MA_TWSE.index.name ='Moving Average'
MA_TPEx = pd.DataFrame(index=['7MA', '14MA', '30MA', '60MA', '90MA', '180MA', '360MA', '720MA', '1080MA'],columns=StockPrice_TPEx.columns.get_level_values(0)[1:].drop_duplicates())
MA_TPEx.index.name ='Moving Average'
for i in range(0,len(MA_TWSE.columns)):
    for j in range(0,len(MA_TWSE)):
        MA_TWSE.iloc[j,i] = round(((StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+1] * StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+2])/(StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+2].mean())).mean(), 2)
    
for i in range(0,len(MA_TPEx.columns)):
    for j in range(0,len(MA_TPEx)):
        MA_TPEx.iloc[j,i] = round(((StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+1] * StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+2])/(StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+2].mean())).mean(), 2)

In [1135]:
MA_TWSE

,台泥(1101),亞泥(1102),嘉泥(1103),環泥(1104),幸福(1108),信大(1109),東泥(1110),味全(1201),味王(1203),大成(1210),...,宏全(9939),信義(9940),裕融(9941),茂順(9942),好樂迪(9943),新麗(9944),潤泰新(9945),三發地產(9946),佳龍(9955),世紀鋼(9958)
Moving Average,,,,,,,,,,,,,,,,,,,,,
7MA,38.08,44.17,20.02,27.91,15.26,18.59,19.45,20.67,41.33,53.59,...,105.59,29.93,196.07,157.69,86.31,23.04,35.23,15.05,25.65,104.74
14MA,37.95,44.21,19.99,28.11,15.41,18.29,19.38,20.51,39.64,53.22,...,102.69,29.71,193.34,157.02,81.69,22.48,35.15,14.78,26.01,102.93
30MA,38.57,44.1,20.11,28.49,16.09,17.97,19.06,20.34,39.49,52.73,...,100.98,29.37,190.08,150.97,80.95,22.27,35.33,14.35,27.09,108.61
60MA,38.0,43.66,19.78,27.67,15.38,17.54,18.74,20.44,38.77,51.86,...,100.22,29.08,183.63,142.4,79.88,21.96,36.39,13.84,26.0,106.9
90MA,37.68,43.34,19.96,27.21,14.84,17.36,18.63,20.3,37.99,50.87,...,97.9,28.5,178.28,137.45,77.81,21.81,38.22,13.68,25.39,104.61
180MA,35.74,41.86,19.31,26.22,14.51,16.64,18.48,19.84,35.98,49.15,...,91.66,28.61,163.72,122.77,75.38,21.4,43.89,13.38,24.12,93.97
360MA,38.53,43.13,19.52,24.62,14.08,18.43,19.66,20.92,34.61,50.49,...,86.0,32.36,180.78,118.07,69.78,21.67,57.77,13.59,25.72,104.6
720MA,42.93,44.95,20.92,23.69,13.94,22.25,19.37,21.74,34.58,51.27,...,76.56,31.38,161.05,106.01,66.37,22.25,56.18,15.89,24.4,112.54
1080MA,42.48,44.18,19.57,22.84,13.61,20.63,19.1,24.14,31.42,46.96,...,70.57,30.95,148.36,101.87,65.39,22.38,54.46,20.42,22.82,101.29


In [1136]:
MA_TPEx

,茂生農經(1240),其祥-KY(1258),安心(1259),德麥(1264),漢來美食(1268),台翰(1336),精華(1565),濱川(1569),力肯(1570),新麥(1580),...,國統(8936),合騏(8937),明安(8938),關中(8941),森鉅(8942),琉園(9949),萬國通(9950),皇田(9951),邁達康(9960),有益(9962)
Moving Average,,,,,,,,,,,,,,,,,,,,,
7MA,45.69,26.87,76.68,282.08,159.85,24.42,238.56,23.15,18.87,131.75,...,27.7,19.81,80.95,57.3,70.31,15.92,14.25,73.51,30.89,19.48
14MA,45.52,26.83,75.85,282.66,159.58,24.61,238.38,22.8,18.74,126.9,...,27.21,19.34,79.91,56.3,69.98,15.73,14.05,72.8,30.8,19.66
30MA,45.76,26.82,75.22,279.94,155.2,25.69,244.34,23.47,18.62,121.25,...,27.62,18.68,85.01,57.24,69.16,17.31,14.0,72.14,30.77,20.06
60MA,46.49,26.71,75.82,275.65,149.08,27.46,254.89,24.77,18.36,116.33,...,29.03,17.92,101.88,59.38,68.35,18.41,13.75,72.0,30.01,21.32
90MA,46.96,26.32,77.7,273.47,147.11,28.47,255.5,24.49,18.18,112.81,...,28.88,17.65,102.41,58.81,66.91,18.99,13.62,70.94,29.08,20.68
180MA,45.89,24.68,75.73,267.95,142.31,27.48,247.92,24.48,17.61,107.21,...,28.0,16.76,101.25,56.3,64.17,19.16,12.9,67.35,27.6,19.47
360MA,59.05,24.69,73.03,271.73,139.99,26.49,266.28,25.74,21.72,105.03,...,26.72,15.58,94.31,54.34,65.64,16.5,12.28,69.4,27.23,19.44
720MA,58.83,24.01,70.65,268.46,138.76,23.06,308.45,28.76,20.08,98.85,...,28.25,16.2,79.14,52.12,66.03,14.29,11.31,87.69,27.94,19.45
1080MA,55.89,23.51,70.56,247.12,137.64,22.09,348.94,26.43,19.68,100.53,...,27.36,16.12,75.1,50.58,66.3,13.4,12.02,88.79,28.55,19.28


In [1137]:
temp = [['<7MA', '<14MA', '<30MA', '<60MA', '<90MA', '<180MA', '<360MA', '<720MA', '<1080MA'], ['>3', '>6', '>9']]
Result_TWSE = pd.DataFrame(columns=pd.MultiIndex.from_product(temp))
Result_TPEx = pd.DataFrame(columns=pd.MultiIndex.from_product(temp))
def Judgment(condition):
  global Result_TWSE
  global Result_TPEx
  if (condition == '7MA'):
    x = 0
  elif (condition == '14MA'):
    x = 1
  elif (condition == '30MA'):
    x = 2
  elif (condition == '60MA'):
    x = 3
  elif (condition == '90MA'):
    x = 4
  elif (condition == '180MA'):
    x = 5
  elif (condition == '360MA'):
    x = 6
  elif (condition == '720MA'):
    x = 7
  elif (condition == '1080MA'):
    x = 8
  Pass_0 = []
  Pass_1 = []
  Pass_2 = []
  for i in range(0,len(list(MA_TWSE))):
    if (MA_TWSE.iloc[x,i] > StockPrice_TWSE.iloc[-1,i*4+1]):
        if ((StockPrice_TWSE.iloc[-1,i*4+4] >= 3) and (StockPrice_TWSE.iloc[-1,i*4+4] < 6)):
            Pass_0.append(MA_TWSE.columns[i])
        elif ((StockPrice_TWSE.iloc[-1,i*4+4] >= 6) and (StockPrice_TWSE.iloc[-1,i*4+4] < 9)):
            Pass_1.append(MA_TWSE.columns[i])
        elif ((StockPrice_TWSE.iloc[-1,i*4+4] >= 9)):
            Pass_2.append(MA_TWSE.columns[i])
  Result_TWSE.iloc[:,x*3] = pd.Series(Pass_0)
  Result_TWSE.iloc[:,x*3+1] = pd.Series(Pass_1)
  Result_TWSE.iloc[:,x*3+2] = pd.Series(Pass_2)
  
  Pass_0 = []
  Pass_1 = []
  Pass_2 = []
  for i in range(0,len(list(MA_TPEx))):
    if (MA_TPEx.iloc[x,i] > StockPrice_TPEx.iloc[-1,i*4+1]):
        if ((StockPrice_TPEx.iloc[-1,i*4+4] >= 3) and (StockPrice_TPEx.iloc[-1,i*4+4] < 6)):
            Pass_0.append(MA_TPEx.columns[i])
        elif ((StockPrice_TPEx.iloc[-1,i*4+4] >= 6) and (StockPrice_TPEx.iloc[-1,i*4+4] < 9)):
            Pass_1.append(MA_TPEx.columns[i])
        elif ((StockPrice_TPEx.iloc[-1,i*4+4] >= 9)):
            Pass_2.append(MA_TPEx.columns[i])
  Result_TPEx.iloc[:,x*3] = pd.Series(Pass_0)
  Result_TPEx.iloc[:,x*3+1] = pd.Series(Pass_1)
  Result_TPEx.iloc[:,x*3+2] = pd.Series(Pass_2)

In [1138]:
for i in range(0,len(MA_TWSE.index)):
    Judgment(MA_TWSE.index[i])
for i in range(0,len(MA_TPEx.index)):
    Judgment(MA_TPEx.index[i])
#Result_TWSE = Result_TWSE.reindex(sorted(Result_TWSE.columns), axis=1)
#Result_TPEx = Result_TPEx.reindex(sorted(Result_TPEx.columns), axis=1)

In [1139]:
Result_TWSE

<7MA                              <14MA                         \
            >3           >6         >9         >3         >6          >9   
0     亞泥(1102)     環泥(1104)   力山(1515)   亞泥(1102)   環泥(1104)    力山(1515)   
1     幸福(1108)     達新(1315)   台玻(1802)   幸福(1108)   達新(1315)    台玻(1802)   
2     信大(1109)     昭輝(1339)   冠軍(1806)   卜蜂(1215)  華友聯(1436)   聯上發(2537)   
3     卜蜂(1215)  富林-KY(1341)  大成鋼(2027)   統一(1216)   聚隆(1466)    志信(2611)   
4     統一(1216)    華友聯(1436)   燦坤(2430)  愛之味(1217)   聚陽(1477)  台驊投控(2636)   
..         ...          ...        ...        ...        ...         ...   
212  慶豐富(9935)          NaN        NaN        NaN        NaN         NaN   
213   全國(9937)          NaN        NaN        NaN        NaN         NaN   
214   信義(9940)          NaN        NaN        NaN        NaN         NaN   
215   裕融(9941)          NaN        NaN        NaN        NaN         NaN   
216   茂順(9942)          NaN        NaN        NaN        NaN         NaN   

        <30MA                          <60MA  ...    <180MA     <360MA  \
           >3        >6        >9         >3  ...        >9         >3   
0    亞泥(1102)  環泥(1104)  年興(1451)   幸福(1108)  ...  泰山(1218)   福壽(1219)   
1    幸福(1108)  達新(1315)  力山(1515)   卜蜂(1215)  ...  年興(1451)   佳格(1227)   
2    卜蜂(1215)  得力(1464)  台玻(1802)  愛之味(1217)  ...  力山(1515)  大統益(1232)   
3    統一(1216)  聚隆(1466)  冠軍(1806)   福壽(1219)  ...  台玻(1802)   宏亞(1236)   
4    福壽(1219)  聚陽(1477)  長榮(2603)  福懋油(1225)  ...  冠軍(1806)   台塑(1301)   
..        ...       ...       ...        ...  ...       ...        ...   
212       NaN       NaN       NaN        NaN  ...       NaN        NaN   
213       NaN       NaN       NaN        NaN  ...       NaN        NaN   
214       NaN       NaN       NaN        NaN  ...       NaN        NaN   
215       NaN       NaN       NaN        NaN  ...       NaN        NaN   
216       NaN       NaN       NaN        NaN  ...       NaN        NaN   

                            <720MA                        <1080MA            \
            >6        >9        >3        >6         >9        >3        >6   
0     達新(1315)  泰山(1218)  亞泥(1102)  怡華(1456)   力山(1515)  亞泥(1102)  怡華(1456)   
1    華友聯(1436)  年興(1451)  信大(1109)  得力(1464)   台玻(1802)  信大(1109)  得力(1464)   
2     怡華(1456)  力山(1515)  福壽(1219)  聚隆(1466)   冠軍(1806)  福壽(1219)  聚隆(1466)   
3     聚隆(1466)  台玻(1802)  佳格(1227)  川飛(1516)  聯發科(2454)  佳格(1227)  川飛(1516)   
4     川飛(1516)  冠軍(1806)  宏亞(1236)  日馳(1526)   瑞軒(2489)  宏亞(1236)  日馳(1526)   
..         ...       ...       ...       ...        ...       ...       ...   
212        NaN       NaN       NaN       NaN        NaN       NaN       NaN   
213        NaN       NaN       NaN       NaN        NaN       NaN       NaN   
214        NaN       NaN       NaN       NaN        NaN       NaN       NaN   
215        NaN       NaN       NaN       NaN        NaN       NaN       NaN   
216        NaN       NaN       NaN       NaN        NaN       NaN       NaN   

                
            >9  
0     力山(1515)  
1     台玻(1802)  
2     冠軍(1806)  
3     瑞軒(2489)  
4    聯上發(2537)  
..         ...  
212        NaN  
213        NaN  
214        NaN  
215        NaN  
216        NaN  

[217 rows x 27 columns]

In [1140]:
Result_TPEx

<7MA                            <14MA                        \
              >3         >6         >9         >3         >6         >9   
0       安心(1259)   正德(2641)   大甲(2221)   德麥(1264)   正德(2641)   倍微(6270)   
1       精華(1565)   滿心(2916)   泰博(4736)   祺驊(1593)   矽瑪(3511)  白紗科(8401)   
2      宏佳騰(1599)  波若威(3163)   華祺(5015)  光洋科(1785)   先益(3531)   大田(8924)   
3       生泰(1777)  台星科(3265)  中菲行(5609)   六角(2732)  友威科(3580)   明安(8938)   
4      光洋科(1785)   鈊象(3293)   倍微(6270)   揚秦(2755)   映興(3597)   有益(9962)   
..           ...        ...        ...        ...        ...        ...   
151  保綠-KY(8423)        NaN        NaN        NaN        NaN        NaN   
152     華研(8446)        NaN        NaN        NaN        NaN        NaN   
153     時報(8923)        NaN        NaN        NaN        NaN        NaN   
154     北基(8927)        NaN        NaN        NaN        NaN        NaN   
155     國統(8936)        NaN        NaN        NaN        NaN        NaN   

         <30MA                           <60MA  ...    <180MA     <360MA  \
            >3         >6        >9         >3  ...        >9         >3   
0     精華(1565)   濱川(1569)  大甲(2221)   精華(1565)  ...  由田(3455)   精華(1565)   
1     祺驊(1593)   正德(2641)  由田(3455)   祺驊(1593)  ...  泰博(4736)   力肯(1570)   
2     生泰(1777)  港建*(3093)  皇龍(3512)   川寶(1595)  ...  力士(4923)   川寶(1595)   
3    光洋科(1785)   大綜(3147)  力士(4923)  宏佳騰(1599)  ...  華祺(5015)  宏佳騰(1599)   
4     晉椿(2064)  台嘉碩(3221)  華祺(5015)   生泰(1777)  ...  岳豐(6220)  光洋科(1785)   
..         ...        ...       ...        ...  ...       ...        ...   
151  大汽電(8931)        NaN       NaN   國統(8936)  ...       NaN        NaN   
152        NaN        NaN       NaN        NaN  ...       NaN        NaN   
153        NaN        NaN       NaN        NaN  ...       NaN        NaN   
154        NaN        NaN       NaN        NaN  ...       NaN        NaN   
155        NaN        NaN       NaN        NaN  ...       NaN        NaN   

                             <720MA                         <1080MA  \
            >6        >9         >3         >6        >9         >3   
0     濱川(1569)  捷迅(2643)   精華(1565)   濱川(1569)  捷迅(2643)   精華(1565)   
1     謚源(2235)  由田(3455)   力肯(1570)   正德(2641)  由田(3455)   力肯(1570)   
2     正德(2641)  泰博(4736)   祺驊(1593)  港建*(3093)  皇龍(3512)   祺驊(1593)   
3    港建*(3093)  力士(4923)   川寶(1595)   好德(3114)  泰博(4736)   川寶(1595)   
4     好德(3114)  華祺(5015)  宏佳騰(1599)   順達(3211)  力士(4923)  宏佳騰(1599)   
..         ...       ...        ...        ...       ...        ...   
151        NaN       NaN        NaN        NaN       NaN        NaN   
152        NaN       NaN        NaN        NaN       NaN        NaN   
153        NaN       NaN        NaN        NaN       NaN        NaN   
154        NaN       NaN        NaN        NaN       NaN        NaN   
155        NaN       NaN        NaN        NaN       NaN        NaN   

                           
            >6         >9  
0     濱川(1569)   捷迅(2643)  
1     謚源(2235)   泰博(4736)  
2     正德(2641)   力士(4923)  
3    港建*(3093)  中菲行(5609)  
4     好德(3114)   岳豐(6220)  
..         ...        ...  
151        NaN        NaN  
152        NaN        NaN  
153        NaN        NaN  
154        NaN        NaN  
155        NaN        NaN  

[156 rows x 27 columns]

In [1109]:
Result_TWSE.to_excel('Result_TWSE.xlsx')
Result_TPEx.to_excel('Result_TPEx.xlsx')

In [1110]:
def highlight(series):
    return ['color: red' if value > (series.mean()*3)
            else 'color: orange' if value > (series.mean()*2)
            else None for value in series]

In [1111]:
Stock_TWSE_recentvolume = StockPrice_TWSE.xs('Volume', axis = 1, level = 1)[-14:]
columns_volume = list(Stock_TWSE_recentvolume.columns)
Stock_TWSE_recentvolume.style.apply(highlight, subset=columns_volume)